In [3]:
import pandas as pd

# CSV 파일 로드
p_result = pd.read_csv("/mnt/W-PANCDR/src/logs/full_CV_PANCDR/GDSC_results.csv")
w_result = pd.read_csv("/mnt/W-PANCDR/src/logs/full_CV_WANCDR/GDSC_results.csv")
# 'Best_params' 기준으로 groupby 후, 'Test_AUC' 평균 계산
p_grouped = p_result.groupby('Best_params')['Test_AUC'].mean()
w_grouped = w_result.groupby('Best_params')['Test_AUC'].mean()
# 평균 Test_AUC가 높은 상위 5개의 Best_params 선택
p_top_5 = p_grouped.sort_values(ascending=False).head(5)
w_top_5 = w_grouped.sort_values(ascending=False).head(5)
p_top_5.to_csv("/mnt/W-PANCDR/src/logs/full_CV_PANCDR/top_5_results.csv")
w_top_5.to_csv("/mnt/W-PANCDR/src/logs/full_CV_WANCDR/top_5_results.csv")
# 결과 출력
print(p_top_5)
print(w_top_5)


Best_params
{'nz': 100, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.01, 'batch_size': [128, 14]}    0.804502
{'nz': 256, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.001, 'lam': 0.1, 'batch_size': [128, 14]}      0.802544
{'nz': 128, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.001, 'lam': 1, 'batch_size': [128, 14]}        0.802317
{'nz': 100, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.1, 'batch_size': [256, 28]}     0.802222
{'nz': 128, 'd_dim': 100, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.01, 'batch_size': [128, 14]}    0.801734
Name: Test_AUC, dtype: float64
Best_params
{'nz': 128, 'd_dim': 100, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.0001, 'batch_size': [128, 14]}    0.832946
{'nz': 128, 'd_dim': 128, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.0001, 'batch_size': [128, 14]}    0.831610
{'nz': 100, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.0001, 'batch_size': [128, 14]}    0.831136
{'nz': 100, 'd_dim': 256, 'lr': 0.001, 'lr_adv': 0.001, 'lam': 0.0001, 'bat